In [15]:
# Start writing code here...
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import igraph

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [16]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
os.listdir("../data")

['.DS_Store',
 'description.txt',
 'input_wagon.xlsx',
 'metrics.csv.gz',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'output_wagon.xlsx',
 'sources.csv.gz',
 'standplaces.csv.gz',
 'wagon.zip',
 'wagon_mode_compat.csv.gz']

In [124]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz") # +
sources = pd.read_csv("../data/sources.csv.gz") # +
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz") #+

In [179]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.abstractions import *

In [180]:
roads_graph, name2vertex = get_roads_graph(metrics, debug=False)

In [181]:
roads_graph.es[0]['info'], roads_graph.es[0]['distance'], roads_graph.es[0]['time']

([(100, 104), (200, 138), (305, 355)], 9, 5)

In [182]:
roads_graph.es['weight'] = roads_graph.es['distance']

In [183]:
# dijkstra_paths = roads_graph.shortest_paths_dijkstra(weights="weight")

In [184]:
# np.min(dijkstra_paths), np.max(dijkstra_paths), np.mean(dijkstra_paths), np.median(dijkstra_paths)

In [188]:
gr = init_stations(sources, name2vertex)


In [187]:
gr[0].vs[0]['info']

{2070: [(1, 50), (1, 50)],
 2071: [(1, 50)],
 2135: [(1, 50)],
 2062: [(1, 50)],
 2155: [(1, 50)]}

In [137]:
gr, wagons = get_wagon_graph(wagon_mode_compat)

In [145]:
def create_order_graph(orders, name2vertex):
    columns = [
        "OrderNum",
        "Start",
        "Finish",
        #"StartDate",
        "Dur",
        "MinUnit",
        "MaxUnit",
        "Tariff",
        "NeedWagonModel",
        "ShortagePenalty"
    ]
    vertices = np.unique(np.concatenate([
        orders.Start.values,
        orders.Finish.values
    ]))
    for v in vertices:
        v = str(v)
        if v not in name2vertex:
            index = len(name2vertex)
            name2vertex[v] = index
    graph = Graph()
    graph.add_vertices(len(name2vertex))
    edges = []
    for start, finish in orders[["Start", "Finish"]].values:
        start = str(start)
        finish = str(finish)
        u = name2vertex[start]
        v = name2vertex[finish]
        edges.append((u, v))
    # print(edges)
    graph.add_edges(edges)
    graph['order_num'] = orders["OrderNum"].values
    graph['dur'] = orders["Dur"].values
    graph['min_unit'] = orders["MinUnit"].values
    graph['max_unit'] = orders["MaxUnit"].values
    graph['tariff'] = orders["Tariff"].apply(lambda xs: [int(x) for x in xs.split(":")]).values
    graph["wagon_model"] = orders["NeedWagonModel"].values
    graph["penalty"] = orders["ShortagePenalty"].values
    return graph
    
def build_orders(orders, name2vertex):
    columns = [
        "OrderNum",
        "Start",
        "Finish",
        "StartDate",
        "Dur",
        "MinUnit",
        "MaxUnit",
        "Tariff",
        "NeedWagonModel",
        "ShortagePenalty"
    ]
    graphs = dict()
    for start_date, rows in orders7[columns].groupby("StartDate"):
        graph = create_order_graph(rows, name2vertex)
        graphs[start_date] = graph
    # return start_date, rows
    return graphs

In [146]:
graphs = build_orders(orders7, name2vertex)

In [148]:
graphs[0]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>